# Material ID : mp-3056

In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pymatgen.core.lattice import Lattice
from jupyter_jsmol import JsmolView
from ipywidgets import Layout, widgets, interact
from IPython.display import display
import json
# An optional utility to display a progress bar
# for long-running loops. `pip install tqdm`.
from tqdm import tqdm

In [2]:
api_key = "IDG8oerg7OP5AeRfbNjUZBb1Ad4Qke2l"
material_id = "mp-3056"

mpr = MPRester(api_key)

structure = mpr.get_structure_by_material_id(material_id)
el_bs = mpr.get_bandstructure_by_material_id(material_id)
el_dos = mpr.get_dos_by_material_id(material_id)
ph_bs = mpr.get_phonon_bandstructure_by_material_id(material_id)
ph_dos = mpr.get_phonon_dos_by_material_id(material_id)

conv_struc = SpacegroupAnalyzer(structure).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Télécharger le fichier .cif

In [3]:
from pymatgen.io.cif import CifWriter

CifWriter(structure).write_file(material_id+".cif")

# Déterminer les vecteurs de bases des ses réseaux direct et réciproque

In [4]:
primitive_struct  = structure

In [5]:
view = quick_view(primitive_struct, "packed", conventional = False) # Représentation interactive de la structure cristalline.
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [6]:
view = quick_view(conv_struc, "packed", conventional = True) # Représentation interactive de la structure cristalline.
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [7]:
length_a = primitive_struct.lattice.a
basis_vectors = primitive_struct.lattice
reciprocal_vectors = primitive_struct.lattice.reciprocal_lattice

print("For the primitive structure")
print("lenght a :", length_a)
print("\ndirect network basis vectors :") 
print(basis_vectors.matrix)
print("\nreciprocal network basis vectors :")
print(reciprocal_vectors.matrix)

For the primitive structure
lenght a : 5.791879482140288

direct network basis vectors :
[[ 3.20535534 -0.01296672  4.82404363]
 [ 1.44694452  2.86021434  4.82404363]
 [-0.021186   -0.01296672  5.79182686]]

reciprocal network basis vectors :
[[ 1.94881441e+00 -9.94148613e-01  4.90289783e-03]
 [ 1.47071244e-03  2.18773941e+00  4.90328242e-03]
 [-1.62440293e+00 -9.94148167e-01  1.07666890e+00]]


In [8]:
length_a = conv_struc.lattice.a
basis_vectors = conv_struc.lattice
reciprocal_vectors = conv_struc.lattice.reciprocal_lattice

print("For the conventionnal structure")
print("lenght a :", length_a)
print("\ndirect network basis vectors :") 
print(basis_vectors.matrix)
print("\nreciprocal network basis vectors :")
print(reciprocal_vectors.matrix)

For the conventionnal structure
lenght a : 3.3685574822914273

direct network basis vectors :
[[ 1.68427874 -2.91725635  0.        ]
 [ 1.68427874  2.91725635  0.        ]
 [ 0.          0.         16.36677463]]

reciprocal network basis vectors :
[[ 1.86524509 -1.07689976  0.        ]
 [ 1.86524509  1.07689976  0.        ]
 [ 0.          0.          0.38389881]]


Structure primitive :
- C'est la plus petite maille possible qui, répétée par translation, permet de reconstituer tout le cristal.
- Elle est plus compacte, avec le nombre minimal d'atomes.
- Utile pour les calculs théoriques et l'étude des propriétés fondamentales du réseau réciproque.

Structure conventionnelle :
- Elle est choisie pour mieux visualiser la symétrie et correspond souvent aux descriptions standard des systèmes cristallins.
- Elle peut contenir plusieurs fois la maille primitive mais respecte mieux les axes de symétrie du système cristallin.
- Plus pratique pour identifier le système cristallin, le type de maille et le groupe ponctuel.

"Un solide cristallin est un solide à structure régulière et périodique, formé d'un empilement ordonné d'un grand nombre d’atomes, de molécules ou d’ions. L’empilement ordonné est appelé réseau ponctuel. Pour une structure donnée le réseau est défini en se donnant un systèmes d’axes et en repérant toutes les positions identiques par rapport à l’origine." (slides 15-16)

Le réseau direct fait référence à la structure spatiale des atomes dans le cristal tandis que le réseau réciproque est une représentation mathématique du réseau direct. Il est obtenu grâce à la relation suivante (condition de Laue, slide 158) :
\begin{equation}
    e^{i\textbf{K}\cdot\textbf{R}}=1
\end{equation}

Propriétés des réseaux direct et réciproque à partir de la slide 159.

# Déterminer son type de maille

In [9]:
cell_type = SpacegroupAnalyzer(conv_struc).get_space_group_symbol()[0]
print("cell type :", cell_type)

cell type : R


# Déterminer son système cristallin

In [10]:
crystal_system = SpacegroupAnalyzer(conv_struc).get_crystal_system()
print("cristal system :", crystal_system)

cristal system : trigonal


En 3D, il existe 14 types de réseaux de Bravais différents. On les classe selon 7 systèmes cristallins et 4 types de mailles. Tous ces réseaux sont repris sur la slide 39.

# Déterminer son groupe ponctuel et son groupe spatial

In [11]:
point_group = SpacegroupAnalyzer(conv_struc).get_point_group_symbol()
print("point group :", point_group)

point group : -3m


In [12]:
space_group = SpacegroupAnalyzer(conv_struc).get_space_group_symbol()
print("space group :", space_group)

space group : R-3m


Les groupes ponctuels servent à décrire les relations de symmétrie au sein d'un réseau grâce aux opérations ponctuelles. Avec les 4 opérations de symétrie ponctuelle ("symétries qui laissent invariant au moins un point de l’espace" = rotation / réflexion / inversion / roto-inversion, slide 48) et selon le système crystallin observé, on peut créer 32 différents groupes ponctuels en 3D. Ils sont repris à la slide 59.

Pour les groupes spatiaux, il s'agit globalement de la même chose étendu à un concept un peu plus large puisqu'il inclut également les opérations de symétrie spatiale ("symétrie qui résultent de la combinaison d’une opération de symétrie ponctuelle et d’une translation" = réflexion avec glissement / rotation hélicoïdale, slide 67). Il existe, en 3D, 230 groupes spatiaux. Quelques-uns sont repris à la slide 78.

# Conclusion

Pour résumer, le matériau NaTlO₂ cristallise dans un système cristallin trigonal. Sa maille conventionnelle est de type rhomboédrique (R), et son groupe d’espace est R-3m. Le groupe ponctuel associé est -3m. Cela implique la présence d’un plan miroir ainsi qu’une roto-inversion d’ordre 3, c’est-à-dire un axe de symétrie d’ordre 3 combiné à une inversion.

Les vecteurs de base du réseau direct pour la structure primitive sont :

\begin{align}
a_1 = ( 3.2053, -0.0130, 4.8240) && a_2 = ( 1.4469,  2.8602, 4.8240) && a_3 = (-0.0212, -0.0130, 5.7918)
\end{align}

Les vecteurs de base du réseau réciproque correspondant sont :

\begin{align}
a^*_1 = ( 1.9488, -0.9941, 0.0049) && a^*_2 = ( 0.0015,  2.1877, 0.0049) && a^*_3 = (-1.6244, -0.9941, 1.0767)
\end{align}

Pour la maille conventionnelle, les longueurs des vecteurs de base du réseau direct sont :

\begin{align}
a_1 = (1.6843, -2.9173,  0) && a_2 = (1.6843,  2.9173,  0) && a_3 = (0     ,  0     , 16.3668)
\end{align}

Les vecteurs de base du réseau réciproque sont :

\begin{align}
a^*_1 = (1.8652, -1.0769, 0     ) && a^*_2 = (1.8652,  1.0769, 0     ) && a^*_3 = (0     ,  0     , 0.3839)
\end{align}


Après vérification, les vecteurs du réseau réciproque sont corrects vis-à-vis du réseau direct car la relation $a \cdot a^* = 2\pi$ et $b \cdot a^* = 0$ est respectée.

Ainsi, le matériau NaTlO₂ possède une structure trigonal avec une maille conventionnelle rhomboédrique et un réseau de Bravais de type R. Son groupe d’espace R-3m et son groupe ponctuel -3m impliquent une symétrie caractéristique des matériaux cristallins trigonaux.